In [1]:
import sqlite3

In [2]:
import pandas as pd
df = pd.read_feather("base_train_data")

In [3]:
df.columns.values

array(['index', 'Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea',
       'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'G

In [4]:
df.dtypes.values

array([dtype('int64'), dtype('int64'), dtype('int64'), dtype('O'),
       dtype('float64'), dtype('int64'), dtype('O'), dtype('O'),
       dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'),
       dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'),
       dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'),
       dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('O'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('O'),
       dtype('O'), dtype('O'), dtype('O'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'),
       dtype('O'), dtype('int64'), dtype('O'), dtype('int64'), dtype('O'),
       dtype('O'), dtype('float64'), dtype('O'), dtype('int64'),
       dtype('int64'), dtype

In [14]:
pd.api.types.is_string_dtype(df.dtypes.values[3])

True

In [15]:
type(df.dtypes.values[3])

numpy.dtype

In [3]:
import numpy
def np_to_sql_type(dtype:numpy.dtype):
    if pd.api.types.is_string_dtype(dtype):
        return "text"
    elif pd.api.types.is_float_dtype(dtype):
        return "real"
    elif pd.api.types.is_integer_dtype(dtype):
        return "int"
    
def df_to_sql_schema(table_name:str, df:pd.DataFrame):
    dtypes = df.dtypes.values
    column_names= df.columns.values
    sql_types = list(map(np_to_sql_type, df.dtypes.values))
    zz = list(zip(column_names,sql_types))
    pre_schema =[f""" [{i[0]}] {i[1]}""" for i in zz]
    schema  = f"""({", ".join(pre_schema)})"""
    return schema

def df_to_sql(table_name:str, db:str,df:pd.DataFrame):
    schema = df_to_sql_schema(table_name, df)
    with sqlite3.connect(db) as conn:
        conn.execute(f"CREATE TABLE {table_name} {schema}")
        insert_phrase = ", ".join(["?" for i in df.columns.values])
        conn.executemany(f"INSERT INTO {table_name} VALUES ({insert_phrase})",df.values)

In [7]:
df_to_sql("base_data","test.db",df)

In [6]:
with sqlite3.connect("test.db") as conn:
    print(conn.execute("SELECT * FROM base_data LIMIT 5").fetchall())

[(925, 926, 20, 'RL', None, 15611, 'Pave', None, 'IR1', 'Lvl', 'AllPub', 'Corner', 'Gtl', 'NWAmes', 'Norm', 'Norm', '1Fam', '1Story', 5, 6, 1977, 1977, 'Gable', 'CompShg', 'VinylSd', 'VinylSd', 'None', 0.0, 'TA', 'TA', 'PConc', 'Gd', 'TA', 'Av', 'ALQ', 767, 'LwQ', 93, 266, 1126, 'GasA', 'TA', 'Y', 'SBrkr', 1126, 0, 0, 1126, 0, 1, 2, 0, 3, 1, 'Ex', 6, 'Typ', 0, None, 'Attchd', 1977.0, 'RFn', 2, 540, 'TA', 'TA', 'Y', 180, 0, 0, 0, 0, 0, None, None, None, 0, 3, 2008, 'WD', 'Abnorml', 175000), (1391, 1392, 90, 'RL', 65.0, 8944, 'Pave', None, 'Reg', 'Lvl', 'AllPub', 'Inside', 'Gtl', 'NAmes', 'Norm', 'Norm', 'Duplex', '1Story', 5, 5, 1967, 1967, 'Gable', 'CompShg', 'Plywood', 'Plywood', 'None', 0.0, 'TA', 'TA', 'CBlock', 'TA', 'TA', 'No', 'Unf', 0, 'Unf', 0, 1584, 1584, 'GasA', 'TA', 'Y', 'SBrkr', 1584, 0, 0, 1584, 0, 0, 2, 0, 4, 2, 'TA', 8, 'Mod', 0, None, 'Detchd', 1967.0, 'Unf', 3, 792, 'TA', 'TA', 'Y', 0, 152, 0, 0, 0, 0, None, None, None, 0, 4, 2009, 'WD', 'Normal', 124000), (417, 418, 

In [37]:
zz = df_to_sql_schema("base_data", df)

In [77]:
ss =[f""" [{i[0]}] {i[1]}""" for i in zz]

In [78]:
ss[0]

' [index] int'

In [79]:
schema  = f"""({", ".join(ss)})"""

In [84]:
f"CREATE TABLE base_data {schema}"

'CREATE TABLE base_data ( [index] int,  [Id] int,  [MSSubClass] int,  [MSZoning] text,  [LotFrontage] real,  [LotArea] int,  [Street] text,  [Alley] text,  [LotShape] text,  [LandContour] text,  [Utilities] text,  [LotConfig] text,  [LandSlope] text,  [Neighborhood] text,  [Condition1] text,  [Condition2] text,  [BldgType] text,  [HouseStyle] text,  [OverallQual] int,  [OverallCond] int,  [YearBuilt] int,  [YearRemodAdd] int,  [RoofStyle] text,  [RoofMatl] text,  [Exterior1st] text,  [Exterior2nd] text,  [MasVnrType] text,  [MasVnrArea] real,  [ExterQual] text,  [ExterCond] text,  [Foundation] text,  [BsmtQual] text,  [BsmtCond] text,  [BsmtExposure] text,  [BsmtFinType1] text,  [BsmtFinSF1] int,  [BsmtFinType2] text,  [BsmtFinSF2] int,  [BsmtUnfSF] int,  [TotalBsmtSF] int,  [Heating] text,  [HeatingQC] text,  [CentralAir] text,  [Electrical] text,  [1stFlrSF] int,  [2ndFlrSF] int,  [LowQualFinSF] int,  [GrLivArea] int,  [BsmtFullBath] int,  [BsmtHalfBath] int,  [FullBath] int,  [HalfB

# test models fit update

In [17]:
with sqlite3.connect("test.db") as conn:
    cc = conn.execute("SELECT * from models LIMIT 1").fetchone()
    cd = list(cc)

In [20]:
model = cloudpickle.loads(cd[3])

In [26]:
model.coef_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03485934, 0.        , 0.        ,
       0.12582198, 0.05860114, 0.        , 0.        ])

In [27]:
with sqlite3.connect("test.db") as conn:
    cc = conn.execute("SELECT identifier from models").fetchall()
    cd = list(cc)

In [28]:
cd

[('<crawto.baseline_model.BaselineRegressionPrediction object at 0x0000021AF8124E20>',),
 ('DecisionTreeRegressor(ccp_alpha=0.0, criterion="mse", max_depth=None, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, presort="deprecated", random_state=None, splitter="best")',),
 ('ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection="cyclic", tol=0.0001, warm_start=False)',),
 ('GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion="friedman_mse", init=None, learning_rate=0.1, loss="ls", max_depth=3, max_features=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, n_iter_no_change=None, presort="deprecated", random_state=None, subsample=1.0, tol=0.0

# test prediction table

In [7]:
query = (
            """SELECT pickled_model, identifier FROM models LIMIT 1"""
        )

In [12]:
import sqlite3
import cloudpickle
with sqlite3.connect("test.db") as conn:
    model, identifier = conn.execute(query).fetchone()
    model = cloudpickle.loads(model)

In [14]:
model, identifier

(ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
            max_iter=1000, normalize=False, positive=False, precompute=False,
            random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
 'ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5, max_iter=1000, normalize=False, positive=False, precompute=False, random_state=None, selection="cyclic", tol=0.0001, warm_start=False)')

# valid target

In [1]:
import pandas as pd
df = pd.read_csv("transformed_valid.df")

In [4]:
df.shape[0]

365

In [5]:
import numpy as np

In [11]:
np.ones_like(df.index.values) * .75

array([0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.